In [1]:
import openpyxl
from pathlib import Path
import os
from datetime import datetime
from configparser import ConfigParser
import psycopg2
from psycopg2.extensions import AsIs
import pyprojroot
import re
import pandas as pd

Load functions from `lib` folder, we will use a function to read db credentials and one for batch insert and updates:

In [2]:
from lib.parseparams import read_dbparams


Define path to workbooks

In [3]:
repodir = pyprojroot.find_root(pyprojroot.has_dir(".git"))

Database credentials are stored in a database.ini file

In [4]:
dbparams = read_dbparams(repodir / 'secrets' / 'database.ini', section='aws-lght-sl')

In [46]:
# connect to the PostgreSQL server
print('Connecting to the PostgreSQL database...')
conn = psycopg2.connect(**dbparams)
cur = conn.cursor()
updated_rows=0

Connecting to the PostgreSQL database...


In [47]:
qrystr="""
UPDATE form.quadrat_samples
SET seedbank=%s
WHERE
%s=ANY(comments)
AND seedbank is NULL
"""

In [48]:
mtchs = [
    ("Soil-persistent","seedbank written as persistent soil"),
    ("Soil-persistent","seedbank written as soil persistent"),
    ("Non-canopy","seedbank written as non canopy")
]

In [49]:
for mtch in mtchs:
    qry = cur.mogrify(qrystr, mtch)
    cur.execute(qry)
    updated_rows = updated_rows + cur.rowcount 

In [50]:
qrystr="""
UPDATE form.quadrat_samples
SET resprout_organ=%s
WHERE
%s=ANY(comments)
AND resprout_organ is NULL
"""

In [51]:
mtchs = [
    ("Short rhizome","resprout organ written as rhizome short"),
    ("Basal","resprout organ written as basal stems"),
    ("Tussock","resprout organ written as none/tussock")
]
# ("???","resprout organ written as rhizome long"),

In [52]:
for mtch in mtchs:
    qry = cur.mogrify(qrystr, mtch)
    cur.execute(qry)
    updated_rows = updated_rows + cur.rowcount 

In [53]:
qry="""SELECT seedbank, resprout_organ, count(*) FROM form.quadrat_samples GROUP BY seedbank, resprout_organ;"""
cur.execute(qry)
res = cur.fetchall()
res

[('Soil-persistent', 'Lignotuber', 496),
 ('Non-canopy', 'Tuber', 301),
 ('Non-canopy', 'Apical', 2),
 ('Soil-persistent', 'Short rhizome', 475),
 ('Transient', 'Lignotuber', 1),
 ('Non-canopy', 'Tussock', 678),
 (None, None, 1560),
 ('Soil-persistent', 'Apical', 4),
 ('Canopy', 'Tussock', 1),
 ('Canopy', 'Lignotuber', 989),
 ('Non-canopy', 'Stolon', 310),
 ('Non-canopy', 'None', 611),
 ('Non-canopy', 'Basal', 1381),
 ('Transient', 'None', 164),
 (None, 'Tussock', 11),
 ('Canopy', 'None', 113),
 ('Soil-persistent', None, 1374),
 ('Transient', 'Basal', 47),
 ('Non-canopy', 'Epicormic', 23),
 ('Canopy', 'Basal', 4),
 ('Soil-persistent', 'Tuber', 74),
 ('Soil-persistent', 'None', 6142),
 ('Transient', None, 548),
 ('Soil-persistent', 'Tussock', 565),
 ('Soil-persistent', 'Epicormic', 5),
 (None, 'Basal', 1),
 (None, 'None', 3),
 (None, 'Lignotuber', 2),
 ('Non-canopy', 'Lignotuber', 60),
 ('Soil-persistent', 'Basal', 683),
 ('Transient', 'Stolon', 3),
 (None, 'Tuber', 17),
 ('Transient', 

In [54]:
conn.commit()        
cur.close()
print("%s rows updated" % (updated_rows))
conn.close()
print('Database connection closed.')

1454 rows updated
Database connection closed.
